In [1]:
import io
import itertools
import os
import pickle
import random

from datetime import datetime
from os import path

from matplotlib import pyplot as plt

import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, LogCosh
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GINConv, GatedGraphConv
from spektral.layers import ops, GlobalSumPool, GlobalAttentionPool
from spektral.utils import batch_iterator, numpy_to_disjoint
from spektral.utils import label_to_one_hot

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
def load_data(amount=None, mode='batch'):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")
    
    A_all, X_all, E_all, y_all = qm9.load_data(return_type='numpy',
                               nf_keys='atomic_num',
                               ef_keys='type',
                               self_loops=True,
                               amount=amount) # None for entire dataset
    # Preprocessing
    if mode == 'batch':
        X_uniq = np.unique(X_all)
        X_uniq = X_uniq[X_uniq != 0]
        E_uniq = np.unique(E_all)
        E_uniq = E_uniq[E_uniq != 0]

        X_all = label_to_one_hot(X_all, X_uniq)
        E_all = label_to_one_hot(E_all, E_uniq)
    elif mode == 'disjoint':
        X_uniq = np.unique([v for x in X_all for v in np.unique(x)])
        E_uniq = np.unique([v for e in E_all for v in np.unique(e)])
        X_uniq = X_uniq[X_uniq != 0]
        E_uniq = E_uniq[E_uniq != 0]

        X_all = [label_to_one_hot(x, labels=X_uniq) for x in X_all]
        E_all = [label_to_one_hot(e, labels=E_uniq) for e in E_all]
    
    return A_all, X_all, E_all, y_all

In [3]:
def sample_from_data(sample_size, A_all, X_all, E_all, y_all, mode='batch'):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")
    if mode == 'batch':
        indices = np.random.choice(X_all.shape[0], sample_size, replace=False)
        A = A_all[indices, :, :]
        X = X_all[indices, :, :]
        E = E_all[indices, :, :, :]
        y = y_all.iloc[indices, :].copy()
        
    if mode == 'disjoint':
        indices = np.random.choice(len(X_all), sample_size, replace=False)
        A = [A_all[i] for i in indices]
        X = [X_all[i] for i in indices]
        E = [E_all[i] for i in indices]
        y = y_all.iloc[indices, :].copy()
    
    return A, X, E, y    

In [4]:
def standardize(y, scaler='power_transformer'):
    if scaler not in ['power_transformer', 'standard_scaler']:
        raise ValueError(f'scaler {scaler} not recognized'
                        'choose "power_transformer" or "standard_scaler"')
    task_to_scaler = dict()
    for task in list(y.columns)[1:]:
        if scaler == 'standard_scaler':
            scaler = StandardScaler()
        else:
            scaler = PowerTransformer()
        y.loc[:, task] = scaler.fit_transform(y[[task]])
        task_to_scaler[task] = scaler
    return task_to_scaler

In [5]:
def get_shape_params(*, A, X, E, mode='batch'):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")
    F = X[0].shape[-1]  # Dimension of node features
    S = E[0].shape[-1]  # Dimension of edge features
    if mode == 'batch':
        N = X.shape[-2]       # Number of nodes in the graphs
        return N, F, S
    if mode == 'disjoint':
        return F, S

In [6]:
def get_input_tensors(*, A, X, E, mode='batch'):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")
    if mode == 'batch':
        N, F, S = get_shape_params(A=A, X=X, E=E, mode=mode)
        X_in = Input(shape=(N, F), name='X_in')
        A_in = Input(shape=(N, N), name='A_in')
        E_in = Input(shape=(N, N, S), name='E_in')

        return X_in, A_in, E_in
    
    if mode == 'disjoint':
        F, S = get_shape_params(A=A, X=X, E=E, mode=mode)
        X_in = Input(shape=(F,), name='X_in')
        A_in = Input(shape=(None,), sparse=True, name='A_in')
        E_in = Input(shape=(S,), name='E_in')
        I_in = Input(shape=(), name='segment_ids_in', dtype=tf.int32)
        
        return X_in, A_in, E_in, I_in

In [7]:
def build_single_task_model(*, A, X, E, learning_rate=1e-3, 
                            conv='ecc', mode='batch',
                            layer_sizes=[128, 256, 512, 512]):  
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")  
    if conv not in ['ecc', 'gin']:
        raise ValueError(f"convolution layer {conv} not recognized; "
                         "choose 'ecc' or 'gin'")
    
    if mode == 'batch':
        X_in, A_in, E_in = get_input_tensors(A=A, X=X, E=E, mode=mode)
    if mode == 'disjoint':
        X_in, A_in, E_in, I_in = get_input_tensors(A=A, X=X, E=E, mode=mode)
        
    gc1_size, gc2_size, pool_size, dense_size = layer_sizes

    if conv == 'ecc':    
        gc1 = EdgeConditionedConv(gc1_size, activation='relu')([X_in, A_in, E_in])
        gc2 = EdgeConditionedConv(gc2_size, activation='relu')([gc1, A_in, E_in])
    if conv == 'gin':
        assert mode == 'disjoint', 'cannot run GIN in batch mode'
        gc1 = GINConv(gc1_size, activation='relu')([X_in, A_in, E_in])
        gc2 = GINConv(gc2_size, activation='relu')([gc1, A_in, E_in])
    if mode == 'batch':
        pool = GlobalAttentionPool(pool_size)(gc2)
    if mode == 'disjoint':
        pool = GlobalAttentionPool(pool_size)([gc2, I_in])
    dense = Dense(dense_size, activation='relu')(pool)
    output = Dense(1)(dense)

    optimizer = Adam(lr=learning_rate)
    loss_fn = MeanSquaredError()
    if mode == 'batch':
        model = Model(inputs=[X_in, A_in, E_in], outputs=output)
        model.compile(optimizer=optimizer, loss=loss_fn)
    if mode == 'disjoint':
        model = Model(inputs=[X_in, A_in, E_in, I_in], outputs=output)
    
    return model, loss_fn

In [8]:
def build_hard_sharing_model(*, A, X, E, num_tasks, 
                             learning_rate=1e-3, conv='ecc', mode='batch', 
                             layer_sizes=[128, 256, 512, 512]):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")  
    if conv not in ['ecc', 'gin']:
        raise ValueError(f"convolution layer {conv} not recognized; "
                         "choose 'ecc' or 'gin'")
    if mode == 'batch':
        X_in, A_in, E_in = get_input_tensors(A=A, X=X, E=E, mode=mode)
    if mode == 'disjoint':
        X_in, A_in, E_in, I_in = get_input_tensors(A=A, X=X, E=E, mode=mode)

    gc1_size, gc2_size, pool_size, dense_size = layer_sizes
    
    if conv == 'ecc':    
        gc1 = EdgeConditionedConv(gc1_size, activation='relu')([X_in, A_in, E_in])
        gc2 = EdgeConditionedConv(gc2_size, activation='relu')([gc1, A_in, E_in])
    if conv == 'gin':
        assert mode == 'disjoint', 'cannot run GIN in batch mode'
        gc1 = GINConv(gc1_size, activation='relu')([X_in, A_in, E_in])
        gc2 = GINConv(gc2_size, activation='relu')([gc1, A_in, E_in])
    if mode == 'batch':
        pool = GlobalAttentionPool(pool_size)(gc2)
    if mode == 'disjoint':
        pool = GlobalAttentionPool(pool_size)([gc2, I_in])
    dense_list = [Dense(dense_size, activation='relu')(pool) 
                  for i in range(num_tasks)]
    output_list = [Dense(1)(dense_layer) for dense_layer in dense_list]

    optimizer = Adam(lr=learning_rate)
    loss_fn = MeanSquaredError()
    if mode == 'batch':
        model = Model(inputs=[X_in, A_in, E_in], outputs=output_list)
        model.compile(optimizer=optimizer, loss=loss_fn)
    if mode == 'disjoint':
        model = Model(inputs=[X_in, A_in, E_in, I_in], outputs=output_list)
    
    return model, loss_fn

In [9]:
def build_soft_sharing_model(*, A, X, E, num_tasks, share_param, 
                             learning_rate=1e-3, conv='ecc', mode='batch', 
                             layer_sizes=[128, 256, 512, 512]):
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")  
    if conv not in ['ecc', 'gin']:
        raise ValueError(f"convolution layer {conv} not recognized; "
                         "choose 'ecc' or 'gin'")
    if mode == 'batch':
        X_in, A_in, E_in = get_input_tensors(A=A, X=X, E=E, mode=mode)
    if mode == 'disjoint':
        X_in, A_in, E_in, I_in = get_input_tensors(A=A, X=X, E=E, mode=mode)
        
    if conv == 'ecc':
        conv_layer = EdgeConditionedConv
    if conv == 'gin':
        conv_layer = GINConv
        
    gc1_size, gc2_size, pool_size, dense_size = layer_sizes

    gc1_list = [conv_layer(gc1_size, activation='relu')([X_in, A_in, E_in]) 
                for i in range(num_tasks)]
    gc2_list = [conv_layer(gc2_size, activation='relu')([gc1, A_in, E_in]) 
                for gc1 in gc1_list]
    if mode == 'batch':
        pool_list = [GlobalAttentionPool(pool_size)(gc2) for gc2 in gc2_list]
    if mode == 'disjoint':
        pool_list = [GlobalAttentionPool(pool_size)([gc2, I_in]) for gc2 in gc2_list]
    dense_list = [Dense(dense_size, activation='relu')(pool) for pool in pool_list]
    output_list = [Dense(1)(dense) for dense in dense_list]

    def loss_fn(y_actual, y_pred):
        avg_layer_diff = 0
        for i, j in itertools.combinations(range(num_tasks), 2):
            for gc in [gc1_list, gc2_list]:
                diff = gc[i].trainable_weights - gc[j].trainable_weights
                avg_layer_diff += mean(square(diff))
        avg_layer_diff /= (num_tasks)*(num_tasks-1)/2  
        return mean(square(y_actual - y_pred)) + share_param*avg_layer_diff

    optimizer = Adam(lr=learning_rate)
    if mode == 'batch':
        model = Model(inputs=[X_in, A_in, E_in], outputs=output_list)
        model.compile(optimizer=optimizer, loss=loss_fn)
    if mode == 'disjoint':
        model = Model(inputs=[X_in, A_in, E_in, I_in], outputs=output_list)
    
    return model, loss_fn

In [10]:
def generate_model_filename(tasks, conv='ecc', mode='batch', folder_path='demo_models'):
    filename = "".join(sorted(tasks)) + '_' + conv + '_' + mode 
    return path.join(folder_path, f'{filename}.h5')

def generate_task_scaler_filename(task, folder_path='demo_models'):
    return path.join(folder_path, f'{task}_scaler.pkl')

In [11]:
def save_model(model, tasks, task_to_scaler, mode='batch', conv='ecc'):
    model.save_weights(generate_model_filename(tasks, conv=conv, mode=mode))
    for task in tasks:
        scaler_filename = generate_task_scaler_filename(task)
        with open(scaler_filename, 'wb') as f:
            scaler = task_to_scaler[task]
            pickle.dump(obj=scaler, file=f)

def load_hard_sharing_model(*, A, X, E, tasks, conv='ecc', 
                            mode='batch', task_to_scaler=dict()):
    model, _ = build_hard_sharing_model(A=A, X=X, E=E, conv=conv, mode=mode,
                                     num_tasks=len(tasks))
    model.load_weights(generate_model_filename(tasks, conv=conv, mode=mode))
    for task in tasks:
        if task not in task_to_scaler:
            with open(generate_task_scaler_filename(task), 'rb') as f:
                task_to_scaler[task] = pickle.load(f)
    return model, task_to_scaler

In [12]:
def train_multitask_disjoint(model, cluster, *, opt, loss_fn, batch_size, 
                             epochs, A_train, X_train, E_train, y_train, 
                             epoch_num=1, loss_logger=None):
    F, S = get_shape_params(A=A_train, X=X_train, E=E_train, mode='disjoint')
    @tf.function(
        input_signature=(tf.TensorSpec((None, F), dtype=tf.float64),
                         tf.SparseTensorSpec((None, None), dtype=tf.float64),
                         tf.TensorSpec((None, S), dtype=tf.float64),
                         tf.TensorSpec((None,), dtype=tf.int32),
                         tf.TensorSpec((None, len(cluster)), dtype=tf.float64)),
        experimental_relax_shapes=True)
    def train_step(X_, A_, E_, I_, y_):
        with tf.GradientTape() as tape:
            predictions = model([X_, A_, E_, I_], training=True)
            loss = loss_fn(y_, predictions)
            loss += sum(model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    current_batch = 0
    model_loss = 0
    batches_in_epoch = np.ceil(len(A_train) / batch_size)

    print('Fitting model')
    batches_train = batch_iterator([X_train, A_train, E_train, y_train[cluster].values],
                                   batch_size=batch_size, epochs=epochs)
    for b in batches_train:
        X_, A_, E_, I_ = numpy_to_disjoint(*b[:-1])
        A_ = ops.sp_matrix_to_sp_tensor(A_)
        y_ = b[-1]
        outs = train_step(X_, A_, E_, I_, y_)

        model_loss += outs.numpy()
        current_batch += 1
        if current_batch == batches_in_epoch:
            print('Loss: {}'.format(model_loss / batches_in_epoch))
            if loss_logger is not None:
                loss_logger.losses[epoch_num] = model_loss / batches_in_epoch
            model_loss = 0
            current_batch = 0
            epoch_num += 1

In [13]:
def test_multitask_disjoint(model, cluster, *, loss_fn, batch_size, 
                            A_test, X_test, E_test, y_test,
                            task_to_scaler=None, model_data=None):
    print('Testing model')
    model_loss = 0
    batches_in_epoch = np.ceil(len(A_test) / batch_size)
    batches_test = batch_iterator([X_test, A_test, E_test, y_test[cluster].values],
                                  batch_size=batch_size)
    for b in batches_test:
        X_, A_, E_, I_ = numpy_to_disjoint(*b[:-1])
        A_ = ops.sp_matrix_to_sp_tensor(A_)
        y_ = b[3]

        predictions = model([X_, A_, E_, I_], training=False)
        model_loss += loss_fn(y_, predictions)
        
        if all([task_to_scaler, model_data]):
            if len(cluster) == 1:
                predictions = [predictions]
            y_ = np.hsplit(y_, len(cluster))
            for task, batch_pred, batch_actual in zip(cluster, predictions, y_):
                scaler = task_to_scaler[task]
                batch_pred = scaler.inverse_transform(batch_pred)
                batch_actual = scaler.inverse_transform(batch_actual)
                for pred, actual in zip(batch_pred, batch_actual):
                    model_data.add_test(task, pred=pred[0], actual=actual[0])

    model_loss /= batches_in_epoch
    print('Done. Test loss: {}'.format(model_loss))
    
    if model_data:
        model_data.test_loss = model_loss

    return model_loss

In [14]:
def predict_property(prop, mol_id, clusters, *, X_all, A_all, E_all,
                     mode='batch', conv='ecc', model=None, 
                     task_to_scaler=dict()):
    
    if mode not in ['batch', 'disjoint']:
        raise ValueError(f"mode {mode} not recognized; "
                         "choose 'batch' or 'disjoint'")  
    if conv not in ['ecc', 'gin']:
        raise ValueError(f"convolution layer {conv} not recognized; "
                         "choose 'ecc' or 'gin'")
    
    cluster = [c for c in clusters if prop in c][0]
    if model is None:
        model, task_to_scaler = load_hard_sharing_model(A=A_all, 
                                                        X=X_all, 
                                                        E=E_all, 
                                                        tasks=cluster,
                                                        mode=mode, 
                                                        conv=conv, 
                                                        task_to_scaler=task_to_scaler)
    i = mol_id - 1

    # convert shape for batch mode
    if mode == 'batch':
        def wrap(a):
            return a.reshape([1] + list(a.shape))
        x = list(map(wrap, [X_all[i], A_all[i], E_all[i]]))
        cluster_prediction = model.predict(x)       
    
    if mode == 'disjoint':
        X_, A_, E_, I_ = numpy_to_disjoint([X_all[i]], [A_all[i]], [E_all[i]])
        A_ = ops.sp_matrix_to_sp_tensor(A_)
        cluster_prediction = model([X_, A_, E_, I_], training=False)
    
    prediction = cluster_prediction[cluster.index(prop)]
    prediction = task_to_scaler[prop].inverse_transform(prediction)
    return prediction[0][0]

In [15]:
class LossLoggerCallback(Callback):
    def __init__(self):
        self.losses = dict()
        
    def on_epoch_end(self, epoch, logs=None):
        self.losses[epoch] = logs["loss"]

class ModelData:
    def __init__(self, params=dict()):
        self.timestamp = datetime.now()
        self.loss_logger = LossLoggerCallback()
        
        """
        Possible params keys:
        mode: 'batch' or 'disjoint'
        conv: 'ecc' or 'gin'
        single_task: true or false
        cluster: only if single_task is false
        hard_sharing: true or false, only if single_task is false 
        soft_weight: only if hard_sharing is false and single_task is false
        batch_size
        epochs
        num_sampled
        learning_rate
        model_summary
        loss_fn: string of loss function name
        optimizer: string of optimizer name
        layer_sizes: List[Int] [gc1, gc2, pool, dense]
        """
        self.params = params
        
        self.actual = dict() # task -> List[Float]
        self.pred = dict() # task -> List[Float]

        self.test_loss = None
        
    def get_losses(self):
        return self.loss_logger.losses
    
    def add_test(self, task, *, actual, pred):
        if task not in self.actual:
            self.actual[task] = list()
            self.pred[task] = list()
        
        self.actual[task].append(actual)
        self.pred[task].append(pred)
    
    def _make_picklable(self):
        return {'params': self.params, 
                'actual': self.actual, 
                'pred': self.pred, 
                'losses': self.loss_logger.losses,
                'test_loss': self.test_loss}
    
    def serialize(self, dirname='model_data', filename=''):
        if filename == '':
            dt_string = self.timestamp.strftime('%d-%m-%Y_%H-%M-%S')
            filename = path.join(dirname, dt_string + '.pkl')
        with open(filename, 'wb') as file:
            pickle.dump(self._make_picklable(), file)

In [16]:
# if __name__ == '__main__' and '__file__' not in globals():
#     mode = 'disjoint'
#     conv = 'gin'
#     batch_size = 32
#     epochs = 40
#     num_sampled = 30000
#     learning_rate = 1e-2
#     learning_rate_scheduler = 2
#     epochs_per_schedule = 5
#     amount = None
#     scaler = 'power_transformer'
#     loss_fn = MeanAbsoluteError()
#     A_all, X_all, E_all, y_all = load_data(amount=amount, mode=mode)

In [17]:
# if __name__ == '__main__' and '__file__' not in globals(): 
#     A, X, E, y = sample_from_data(num_sampled, 
#                                   A_all, 
#                                   X_all, 
#                                   E_all, 
#                                   y_all, 
#                                   mode=mode)
#     task_to_scaler = standardize(y)

In [18]:
clusters = [['A', 'lumo', 'homo'], 
            ['B', 'r2', 'cv'], 
            ['alpha', 'zpve'], 
            ['C', 'u0', 'u298', 'mu'], 
            ['g298', 'h298']]

In [19]:
# if __name__ == '__main__' and '__file__' not in globals():     
#     A_train, A_test, \
#         X_train, X_test, \
#         E_train, E_test, \
#         y_train, y_test = train_test_split(A, X, E, y, test_size=0.1)

In [20]:
# if __name__ == '__main__' and '__file__' not in globals():
#     print('begin training models')
   
#     tasks = [[task] for cluster in clusters for task in cluster]
#     clusters_alt = [['B', 'g298'], ['alpha', 'zpve'], ['C', 'u0', 'u298'], ['r2', 'cv'], ['h298', 'mu']]
#     tasks_and_clusters = itertools.chain(tasks, clusters, clusters_alt)
#     for cluster, conv in itertools.product(tasks_and_clusters,
#                                            ['ecc', 'gin']):
#         print(f'training {cluster} with {mode} mode on {conv} conv')
        
#         model, _ = build_hard_sharing_model(A=A_train, 
#                                                   X=X_train, 
#                                                   E=E_train, 
#                                                   num_tasks=len(cluster),
#                                                   mode=mode,
#                                                   conv=conv)
#         optimizer = Adam(lr=learning_rate)
#         loss_fn = MeanSquaredError()
        
#         stream = io.StringIO()
#         model.summary(print_fn=lambda x: stream.write(x + '\n'))
#         summary = stream.getvalue()
        
#         params = {'mode': mode, 
#                   'conv': conv,
#                   'batch_size': batch_size,
#                   'epochs': epochs,
#                   'num_sampled': num_sampled,
#                   'learning_rate': learning_rate,
#                   'cluster': cluster,
#                   'hard_sharing': True,
#                   'model_summary': summary,
#                   'loss_fn': type(loss_fn).__name__,
#                   'optimizer': type(optimizer).__name__, 
#                   'learning_rate_scheduler': learning_rate_scheduler, 
#                   'epochs_per_schedule': epochs_per_schedule}
#         model_data = ModelData(params=params)
                
#         if mode == 'batch':
#             # training
#             y_train_cluster = np.hsplit(y_train[cluster].values, len(cluster))
#             model.compile(optimizer=optimizer, 
#                           loss=loss_fn)
#             model.fit(x=[X_train, A_train, E_train], 
#                       y=y_train_cluster,
#                       batch_size=batch_size,
#                       validation_split=0.1,
#                       epochs=epochs,
#                       callbacks=[model_data.loss_logger])
            
#             # testing
#             y_test_cluster = np.hsplit(y_test[cluster].values, len(cluster))
#             model_loss = model.evaluate(x=[X_test, A_test, E_test],
#                                         y=y_test_cluster)
#             print(f"Test loss on {cluster}: {model_loss}")
#             cluster_pred = model.predict([X_test, A_test, E_test])

#         if mode == 'disjoint':
#             # training with learning rate decay
#             for i in range(epochs//epochs_per_schedule):
#                 if i > 0:
#                     learning_rate /= learning_rate_scheduler
#                 optimizer = Adam(lr=learning_rate)
#                 train_multitask_disjoint(model,
#                                          cluster,
#                                          opt=optimizer,
#                                          loss_fn=loss_fn,
#                                          batch_size=batch_size,
#                                          epochs=epochs_per_schedule,
#                                          epoch_num=i*epochs_per_schedule+1,
#                                          A_train=A_train,
#                                          X_train=X_train,
#                                          E_train=E_train,
#                                          y_train=y_train, 
#                                          loss_logger=model_data.loss_logger)
#             # testing
#             model_loss = test_multitask_disjoint(model,
#                                                 cluster,
#                                                 loss_fn=loss_fn,
#                                                 batch_size=batch_size,
#                                                 A_test=A_test,
#                                                 X_test=X_test,
#                                                 E_test=E_test,
#                                                 y_test=y_test)
#             X_, A_, E_, I_ = numpy_to_disjoint(X_test, A_test, E_test)
#             A_ = ops.sp_matrix_to_sp_tensor(A_)
#             cluster_pred = model([X_, A_, E_, I_], training=False)
        
#         if len(cluster) == 1:
#             cluster_pred = [cluster_pred]
            
#         for prop, batch_pred in zip(cluster, cluster_pred):
#             batch_pred = task_to_scaler[prop].inverse_transform(batch_pred)
#             errors = list()
#             for index, pred in zip(y_test.index.values, batch_pred):
#                 actual = y_all.loc[index, prop]
#                 model_data.add_test(prop, actual, pred[0])
        
#         # save_model(model, cluster, task_to_scaler, mode=mode, conv=conv)
#         model_data.serialize()

In [ ]:
def random_hyperparameter_search(trials=50):
    mode = 'batch'
    conv = 'ecc'
    amount = 1000 # None
    A_all, X_all, E_all, y_all = load_data(amount=amount, mode=mode)
    
    for _ in range(trials):
        batch_size = random.choice([16, 32, 64])
        learning_rate = random.choice([1e-2, 1e-3, 1e-4])
        epochs = 5 # random.choice([20, 30, 40])
        layer_sizes = [random.choice([16, 32, 64, 128, 256, 512]), 
               random.choice([16, 32, 64, 128, 256, 512]), 
               random.choice([16, 32, 64, 128, 256, 512]), 
               random.choice([64, 128, 256, 512])]
        num_sampled = 100 # 30000
        scaler = random.choice(['power_transformer', 'standard_scaler'])
        loss_fn = random.choice([MeanAbsoluteError(), MeanSquaredError()])
        soft_sharing = random.random() < 0.5
        share_param = random.choice([1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3])
        optimizer = Adam(lr=learning_rate)
    
        A, X, E, y = sample_from_data(num_sampled,
                                     A_all,
                                     X_all,
                                     E_all,
                                     y_all,
                                     mode=mode)
        task_to_scaler = standardize(y, scaler=scaler)
        
        cluster = ['r2', 'cv']
        A_train, A_test, \
        X_train, X_test, \
        E_train, E_test, \
        y_train, y_test = train_test_split(A, X, E, y, test_size=0.1)
        
        if soft_sharing:
            model, _ = build_soft_sharing_model(A=A_train,
                                                X=X_train,
                                                E=E_train,
                                                num_tasks=len(cluster),
                                                share_param=share_param,
                                                mode=mode,
                                                conv=conv,
                                                layer_sizes=layer_sizes)
        else:
            model, _ = build_hard_sharing_model(A=A_train,
                                                X=X_train,
                                                E=E_train,
                                                num_tasks=len(cluster),
                                                mode=mode,
                                                conv=conv,
                                                layer_sizes=layer_sizes)
        
        stream = io.StringIO()
        model.summary(print_fn=lambda x: stream.write(x + '\n'))
        summary = stream.getvalue()
        
        params = {'mode': mode,
                 'conv': conv,
                 'batch_size': batch_size,
                 'epochs': epochs,
                 'num_sampled': num_sampled,
                 'learning_rate': learning_rate,
                 'cluster': cluster,
                 'hard_sharing': not soft_sharing,
                 'share_param': share_param,
                 'model_summary': summary,
                 'loss_fn': type(loss_fn).__name__,
                 'optimizer': type(optimizer).__name__}
        model_data = ModelData(params=params)
        
        y_train_cluster = np.hsplit(y_train[cluster].values, len(cluster))
        model.compile(optimizer=optimizer, 
                      loss=loss_fn)
        model.fit(x=[X_train, A_train, E_train], 
                  y=y_train_cluster,
                  batch_size=batch_size,
                  validation_split=0.1,
                  epochs=epochs,
                  callbacks=[model_data.loss_logger])
        
        y_test_cluster = np.hsplit(y_test[cluster].values, len(cluster))
        model_loss = model.evaluate(x=[X_test, A_test, E_test],
                                    y=y_test_cluster)
        model_data.test_loss = model_loss
        cluster_pred = model.predict([X_test, A_test, E_test])

        for prop, batch_pred in zip(cluster, cluster_pred):
            batch_pred = task_to_scaler[prop].inverse_transform(batch_pred)
            errors = list()
            for index, pred in zip(y_test.index.values, batch_pred):
                actual = y_all.loc[index, prop]
                model_data.add_test(prop, actual=actual, pred=pred[0])
        
        dirname = 'model_data'
        dt_string = model_data.timestamp.strftime('%d-%m-%Y_%H-%M-%S')
        filename = path.join(dirname, 'hyperparam_search' + dt_string + '.pkl')
        # model_data.serialize(filename=filename)

if __name__ == '__main__' and '__file__' not in globals():
    random_hyperparameter_search(50)

Loading QM9 dataset.
Reading SDF
Epoch 1/5
2/2 [==============================] - 1s 448ms/step - loss: 1.9597 - dense_14_loss: 0.9697 - dense_15_loss: 0.9899 - val_loss: 3.7859 - val_dense_14_loss: 2.0743 - val_dense_15_loss: 1.7116
Epoch 2/5
2/2 [==============================] - 0s 198ms/step - loss: 1.9413 - dense_14_loss: 0.9637 - dense_15_loss: 0.9776 - val_loss: 3.8419 - val_dense_14_loss: 2.0918 - val_dense_15_loss: 1.7500
Epoch 3/5
2/2 [==============================] - 0s 173ms/step - loss: 1.9185 - dense_14_loss: 0.9557 - dense_15_loss: 0.9629 - val_loss: 3.7864 - val_dense_14_loss: 2.0728 - val_dense_15_loss: 1.7136
Epoch 4/5
2/2 [==============================] - 0s 250ms/step - loss: 1.9051 - dense_14_loss: 0.9527 - dense_15_loss: 0.9524 - val_loss: 3.7597 - val_dense_14_loss: 2.0640 - val_dense_15_loss: 1.6956
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 0.8185 - dense_14_loss: 0.3186 - dense_15_loss: 0.4999
Epoch 1/5
3/3 [========================